In [ ]:
# | default_exp classes.DomoPDP

In [ ]:
# | exporti
import asyncio
import datetime as dt
import io
import json

import httpx
import pandas as pd

from nbdev.showdoc import patch_to

from dataclasses import dataclass, field
from enum import Enum, auto

import domolibrary.utils.DictDot as util_dd
import domolibrary.utils.chunk_execution as ce

import domolibrary.routes.pdp as pdp_routes

import domolibrary.client.DomoAuth as dmda
import domolibrary.client.DomoError as de

# from ..utils.chunk_execution import chunk_list
# from . import DomoCertification as dmdc

The PDP class is an example of how domolibrary uses composition.  PDP is a feature of Datasets; however to segment classes, PDP was written as a separate class.  In a future sprint the naming of this class may be modified for clarity.

### PDP Policy Parameter

In [ ]:
# | export
@dataclass
class PDP_Parameter:
    column_name: str
    column_values_ls: list
    operator: str = (
        "EQUALS"
        or "GREATER_THAN"
        or "LESS_THAN"
        or "GREATER_THAN_EQUAL"
        or "LESS_THAN_EQUAL"
        or "BETWEEN"
    )
    ignore_case: bool = True
    type: str = (
        "COLUMN" or "DYNAMIC"
    )  # column sets parameter on data vs dynamic creates on Domo Trusted Attribute

In [ ]:
# | export
@patch_to(PDP_Parameter)
def generate_parameter_simple(obj):

    return pdp_routes.generate_policy_parameter_simple(
        column_name=obj.name,
        type=obj.type,
        column_values_ls=obj.values,
        operator=obj.operator,
        ignore_case=obj.ignoreCase,
    )


@patch_to(PDP_Parameter)
def generate_body_from_parameter(self):

    return pdp_routes.generate_policy_parameter_simple(
        column_name=self.column_name,
        type=self.type,
        column_values_ls=self.column_values_ls,
        operator=self.operator,
        ignore_case=self.ignore_case,
    )

In [ ]:
param = PDP_Parameter(column_name="instance", column_values_ls=["domo-community"])

PDP_Parameter.generate_body_from_parameter(param)

{'type': 'COLUMN',
 'name': 'instance',
 'values': ['domo-community'],
 'operator': 'EQUALS',
 'ignoreCase': True}

### PDP Policy

In [ ]:
# | export
@dataclass
class PDP_Policy:
    dataset_id: str
    filter_group_id: str
    name: str
    # resources: list
    parameters_ls: list[dict]
    user_ls: list[str]
    group_ls: list[str]
    virtual_user_ls: list[str]

    @classmethod
    async def _from_json(cls, obj, auth: dmda.DomoAuth):
        dd = util_dd.DictDot(obj)

        import domolibrary.classes.DomoUser as dmu
        import domolibrary.classes.DomoGroup as dmg

        return cls(
            dataset_id=dd.dataSourceId,
            filter_group_id=dd.filterGroupId,
            name=dd.name,
            # resources=dd.resources,
            parameters_ls=dd.parameters,
            user_ls=(
                await ce.gather_with_concurrency(
                    n=60,
                    *[
                        dmu.DomoUser.get_by_id(user_id=id, auth=auth)
                        for id in dd.userIds
                    ],
                )
                if dd.userIds
                else None
            ),
            group_ls=(
                await ce.gather_with_concurrency(
                    n=60,
                    *[
                        dmg.DomoGroup.get_by_id(group_id=id, auth=auth)
                        for id in dd.groupIds
                    ],
                )
                if dd.groupIds
                else None
            ),
            virtual_user_ls=dd.virtualUserIds,
        )

    @classmethod
    async def upsert_policy(
        cls,
        auth: dmda.DomoAuth,
        dataset_id: str,
        # body sent to the API (uses camelCase instead of snake_case)
        policy_definition: dict,
        debug_api: bool = False,
        debug_prn: bool = False,
    ):

        # print(policy_definition)
        policy_id = policy_definition.get("filterGroupId")
        if policy_id:
            if debug_prn:
                print(f"Updating policy {policy_id} in {auth.domo_instance}")
            res = await pdp_routes.update_policy(
                auth=auth,
                dataset_id=dataset_id,
                policy_id=policy_id,
                body=policy_definition,
                debug_api=debug_api,
            )
            return res
        else:
            if debug_prn:
                print(f"Policy does not exist. Creating policy in {auth.domo_instance}")
            res = await pdp_routes.create_policy(
                auth=auth,
                dataset_id=dataset_id,
                body=policy_definition,
                debug_api=debug_api,
            )
            return res

In [ ]:
# #| export
# @patch_to(PDP_Policy)
# def add_parameter(self: PDP_Policy, parameter_obj):
#     self.parameters.concat(parameter_obj)

In [ ]:
# | exporti
@patch_to(PDP_Policy)
def generate_body_from_policy(
    self: PDP_Policy,
    # params: list[dict] = ''
):
    if not self.parameters_ls:
        raise Exception("generate_body_from_policy: no parameters")

    self.parameters_ls = [
        PDP_Parameter.generate_parameter_simple(param) for param in self.parameters_ls
    ]

    return pdp_routes.generate_policy_body(
        policy_name=self.name,
        dataset_id=self.dataset_id,
        parameters_ls=self.parameters_ls,
        policy_id=self.filter_group_id,
        user_ids=self.user_ls,
        group_ids=self.group_ls,
        virtual_user_ids=self.virtual_user_ls,
    )

### Dataset PDP Policies 

In [ ]:
# | export
class Dataset_PDP_Policies:

    dataset = None  # domo dataset class
    policies: list[PDP_Policy] = None
    auth = None

    def __init__(self, dataset):
        self.dataset = dataset
        self.policies = []

### Get PDP Policies from dataset

In [ ]:
# | exporti
@patch_to(Dataset_PDP_Policies)
async def get_policies(
    self: Dataset_PDP_Policies,
    include_all_rows: bool = True,
    auth: dmda.DomoAuth = None,
    dataset_id: str = None,
    return_raw: bool = False,
    debug_api: bool = False,
):

    dataset_id = dataset_id or self.dataset.id
    auth = auth or self.dataset.auth

    res = await pdp_routes.get_pdp_policies(
        auth=auth,
        dataset_id=dataset_id,
        debug_api=debug_api,
        include_all_rows=include_all_rows,
    )

    if return_raw:
        return res

    if res.status == 200:
        domo_policy = await ce.gather_with_concurrency(
            n=60,
            *[
                PDP_Policy._from_json(policy_obj, auth=auth)
                for policy_obj in res.response
            ]
        )
        self.policies = domo_policy
        return domo_policy

### sample implementation of get_policies

In [ ]:
import os
import domolibrary.classes.DomoDataset as dmd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)
# instantiate dataset class
dataset = dmd.DomoDataset(auth=token_auth, id=os.environ["DOJO_DATASET_ID"])
# instantiate dataset_pdp_policies
dataset_pdp_policies = Dataset_PDP_Policies(dataset=dataset)
# get policies
await dataset_pdp_policies.get_policies(return_raw=False)


# print(dataset.id)
# for res in res.response:
#     print(res['resources'])

[PDP_Policy(dataset_id='04c1574e-c8be-4721-9846-c6ffa491144b', filter_group_id=797, name='All Rows', parameters_ls=None, user_ls=None, group_ls=None, virtual_user_ls=['vu:a8841f70-909a-438e-9853-49783846bf22', 'vu:0ed4455a-35df-4049-8d6d-80341083b2f4', 'vu:566d15dc-9600-48af-b5ad-89cd0ea00cb7', 'vu:ed6853bb-d8c7-4777-be41-0af475fde7ba', 'vu:20e30033-edb3-44f0-a121-5ccf5f8161bc']),
 PDP_Policy(dataset_id='04c1574e-c8be-4721-9846-c6ffa491144b', filter_group_id=1149, name='Test Update Policy Name', parameters_ls=[DictDot(name='objectID', value='000003007', values=['000003007'], type='COLUMN', operator='EQUALS', not=False, ignoreCase=True)], user_ls=[DomoUser(id='308783524', title=None, department=None, display_name='test1', email_address='test1@test.com', role_id=1563101750, avatar_key=None, phone_number=None, web_landing_page=None, web_mobile_landing_page=None, employee_id=None, employee_number=None, hire_date=None, reports_to=None, publisher_domain=None, subscriber_domain=None, virtual_

### sample implementation of generate_body_from_policy

In [ ]:
import os
import domolibrary.classes.DomoDataset as dmd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

# instantiate dataset class
dataset = dmd.DomoDataset(auth=token_auth, id=os.environ["DOJO_DATASET_ID"])

# instantiate dataset_pdp_policies
dataset_pdp_policies = Dataset_PDP_Policies(dataset=dataset)
# get policies
await dataset_pdp_policies.get_policies(return_raw=False)

selected_policy = dataset_pdp_policies.policies[0]

selected_policy


# selected_policy.generate_body_from_policy()

PDP_Policy(dataset_id='04c1574e-c8be-4721-9846-c6ffa491144b', filter_group_id=797, name='All Rows', parameters_ls=None, user_ls=None, group_ls=None, virtual_user_ls=['vu:a8841f70-909a-438e-9853-49783846bf22', 'vu:0ed4455a-35df-4049-8d6d-80341083b2f4', 'vu:566d15dc-9600-48af-b5ad-89cd0ea00cb7', 'vu:ed6853bb-d8c7-4777-be41-0af475fde7ba', 'vu:20e30033-edb3-44f0-a121-5ccf5f8161bc'])

### sample implementation of upsert_policy

In [ ]:
token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

# instantiate dataset class
dataset = dmd.DomoDataset(auth=token_auth, id=os.environ["DOJO_DATASET_ID"])

# instantiate dataset_pdp_policies
dataset_pdp_policies = Dataset_PDP_Policies(dataset=dataset)

# Updating existing policy
new_param = {
    "type": "COLUMN",
    "name": "Country",
    "values": ["Japan", "United States"],
    "operator": "EQUALS",
    "ignoreCase": True,
}

await dataset_pdp_policies.get_policies(return_raw=False)

selected_policy = dataset_pdp_policies.policies[0]
try:
    body = selected_policy.generate_body_from_policy()

    body["parameters"].append(new_param)

    await PDP_Policy.upsert_policy(
        auth=token_auth,
        dataset_id=dataset.id,
        policy_definition=body,
        debug_api=False,
        debug_prn=True,
    )

except Exception as e:
    print(e)

generate_body_from_policy: no parameters


In [ ]:
# upsert_policy, no policy exists so creates new policy
import json

# instantiate dataset class
dataset = dmd.DomoDataset(auth=token_auth, id=os.environ["DOJO_DATASET_ID"])

param = {
    "type": "COLUMN",
    "name": "Country",
    "values": ["Philippines"],
    "operator": "EQUALS",
    "ignoreCase": True,
}

print(param["name"])
new_pdp_policy = PDP_Policy(
    dataset_id=dataset.id,
    filter_group_id=None,
    name="New PDP Policy",
    parameters_ls=[util_dd.DictDot(param)],
    user_ls=["612085674"],
    group_ls=["1259653287"],
    virtual_user_ls=[],
)
body = new_pdp_policy.generate_body_from_policy()
await PDP_Policy.upsert_policy(
    auth=dataset.auth, dataset_id=dataset.id, policy_definition=body
)

Country
Creating policy...


ResponseGetData(status=400, response='Bad Request', is_success=False, parent_class=None, traceback_details=TracebackDetails(function_name='upsert_policy', file_name='/tmp/ipykernel_15375/1590334250.py', function_trail='<module> -> upsert_policy', traceback_stack=[<FrameSummary file /tmp/ipykernel_15375/799330789.py, line 23 in <module>>, <FrameSummary file /tmp/ipykernel_15375/1590334250.py, line 56 in upsert_policy>], parent_class=None))

### Search PDP Policies by name and ID

In [ ]:
# | export
class SearchPDP_NotFound(de.DomoError):
    def __init__(
        self, domo_instance, dataset_id, message="not found", function_name="search_pdp"
    ):

        super().__init__(
            domo_instance=domo_instance,
            entity_id=dataset_id,
            message=message,
            function_name=function_name,
        )


@patch_to(Dataset_PDP_Policies, cls_method=True)
async def search_pdp_policies(
    cls: Dataset_PDP_Policies,
    auth: dmda.DomoAuth,
    search: str,
    dataset_id: str = None,
    search_method: str = "id" or "name",
    is_exact_match: bool = True,
    return_raw: bool = False,
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
):

    all_pdp_policies = await Dataset_PDP_Policies(cls).get_policies(
        auth=auth, dataset_id=dataset_id, debug_api=debug_api
    )

    if return_raw:
        return all_pdp_policies

    if search_method == "name":
        if is_exact_match:
            policy_search = next(
                (policy for policy in all_pdp_policies if policy.name == search), None
            )
            # print(policy_search)

            if not policy_search:
                raise SearchPDP_NotFound(
                    dataset_id=dataset_id,
                    message=f'There is no policy named "{search}" on dataset_id {dataset_id}',
                    domo_instance=auth.domo_instance,
                )

            return policy_search
        else:
            policy_search = [
                policy
                for policy in all_pdp_policies
                if search.lower() in policy.name.lower()
            ]
            if not policy_search:
                raise SearchPDP_NotFound(
                    dataset_id=dataset_id,
                    message=f'There is no policy name containing "{search}" on dataset_id {dataset_id}',
                    domo_instance=auth.domo_instance,
                )

            return policy_search
    else:
        policy_search = next(
            (policy for policy in all_pdp_policies if policy.filter_group_id == search),
            None,
        )

    if not policy_search:
        raise SearchPDP_NotFound(
            dataset_id=dataset_id,
            message=f'There is no policy id "{search}" on dataset_id {dataset_id}',
            domo_instance=auth.domo_instance,
        )

    return policy_search

### sample implementation of search_pdp_policies

In [ ]:
import os
import domolibrary.classes.DomoDataset as dmd

dataset_id = os.environ["DOJO_DATASET_ID"]

# authenticate into instance
token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

# instance of dataset class
dataset = dmd.DomoDataset(auth=token_auth, id=dataset_id)

# instance of pdp_policies for the dataset
dpp = Dataset_PDP_Policies(dataset=dataset)

# pull all policies for that dataset
await dpp.get_policies()

# search policies based on name
try:
    policy = await dpp.search_pdp_policies(
        auth=token_auth,
        dataset_id=dataset.id,
        search="Test 94",
        search_method="name",
        is_exact_match=True,
        debug_api=False,
    )
    print(policy)

except SearchPDP_NotFound as e:
    print(e)

search_pdp: 04c1574e-c8be-4721-9846-c6ffa491144b :There is no policy named "Test 94" on dataset_id 04c1574e-c8be-4721-9846-c6ffa491144b at domo-community


### Delete PDP Policy

In [ ]:
# | export
@patch_to(PDP_Policy)
async def delete_policy(
    self: PDP_Policy,
    auth: dmda.DomoAuth,
    policy_id: str = None,
    dataset_id: str = None,
    debug_api: bool = False,
):

    dataset_id = dataset_id or self.dataset_id
    policy_id = policy_id or self.filter_group_id

    res = await pdp_routes.delete_policy(
        auth=auth, dataset_id=dataset_id, policy_id=policy_id, debug_api=debug_api
    )

    return res

#### Sample implementation of delete_policy

In [ ]:
import os
import domolibrary.classes.DomoDataset as dmd

dataset_id = os.environ["DOJO_DATASET_ID"]

# authenticate into instance
token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

# instance of dataset class
dataset = dmd.DomoDataset(auth=token_auth, id=dataset_id)

# instance of pdp_policies for the dataset
dpp = Dataset_PDP_Policies(dataset=dataset)

# search policies based on name
try:
    policy = await dpp.search_pdp_policies(
        auth=token_auth,
        dataset_id=dataset.id,
        search="Test Policy Name Employee 123",
        search_method="name",
        is_exact_match=True,
        debug_api=False,
    )
    print(policy)

except SearchPDP_NotFound as e:
    print(e)

# delete the policy based on filtergroup id
try:
    await policy.delete_policy(
        auth=token_auth, debug_api=False
    )  # don't need to include policy or dataset since those are already tied with policy class
except Exception as e:
    print(e)

search_pdp: 04c1574e-c8be-4721-9846-c6ffa491144b :There is no policy named "Test Policy Name Employee 123" on dataset_id 04c1574e-c8be-4721-9846-c6ffa491144b at domo-community
name 'policy' is not defined


### Enable/Disable PDP on dataset

In [ ]:
# | export
@patch_to(Dataset_PDP_Policies)
async def toggle_dataset_pdp(
    self: Dataset_PDP_Policies,
    auth: dmda.DomoAuth = None,
    dataset_id: str = None,
    is_enable: bool = True,  # True will enable pdp, False will disable pdp
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
):
    auth = auth or self.dataset.auth

    return await pdp_routes.toggle_pdp(
        auth=auth,
        dataset_id=dataset_id or self.dataset.id,
        is_enable=is_enable,
        debug_api=debug_api,
        session=session,
    )

### sample implementation of toggle_dataset_pdp

In [ ]:
import os
import domolibrary.classes.DomoDataset as dmd

dataset_id = os.environ["DOJO_DATASET_ID"]

# authenticate into instancedataset_id
token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

# instance of dataset class
dataset = dmd.DomoDataset(auth=token_auth, id=dataset_id)

# instance of pdp_policies for the dataset
dpp = Dataset_PDP_Policies(dataset=dataset)

# toggle pdp on dataset
await dpp.toggle_dataset_pdp(
    auth=token_auth, dataset_id=dataset.id, is_enable=False, debug_api=False
)

ResponseGetData(status=200, response={'enabled': False, 'secured': False, 'external': False}, is_success=True, parent_class=None, traceback_details=TracebackDetails(function_name='toggle_dataset_pdp', file_name='/tmp/ipykernel_15375/1954515221.py', function_trail='<module> -> toggle_dataset_pdp', traceback_stack=[<FrameSummary file /tmp/ipykernel_15375/2715666841.py, line 19 in <module>>, <FrameSummary file /tmp/ipykernel_15375/1954515221.py, line 14 in toggle_dataset_pdp>], parent_class=None))

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()